In [1]:
import torch
import torch.nn as nn

<center>
<img src="https://miro.medium.com/max/1200/0*Qn6yUu-y5jbyptAp.png">
    </center>

In [17]:
class Self_attention(nn.Module):
    def __init__(self,embedding_size, heads):
        super(Self_attention,self).init()
        self.embedding_size=embedding_size# get the embedding dimension
        self.heads=heads#the number of heads required
        heads_dim=self.embedding_size//self.heads#dividing the dimesion by the heads count to determine the number 
        #of dimesnions per each head
        assert (self.heads_dim*heads==embedding_size),"Wrong dimensions"# checking the dimesions
        self.keys = nn.Linear(self.heads_dim,self.heads_dim,bias=False)#the keys layer
        self.values = nn.Linear(self.heads_dim,self.heads_dim,bias =False)# the values layer
        self.queries=nn.Linear(self.heads_dim,self.heads_dim,bias=False)#the queries layer
        self.fc_out=nn.Linear(embeddin_size,embedding_size)# the concatenation layer
#===============================================================================================================
    def forward(self,keys,values,query,mask):
        N=query.shape[0]  #number of examples in the query
        value_len,key_len,query_len=values.shape[1],kets.shape[1],query.shape[1]
        #Reshaping the inputs
        values=values.reshape(N,value_len,self.heads,self.heads_dim)
        keys=keys.reshape(N,key_len,self.heads,self.heads_dim)
        query=query.reshape(N,query_len,self.heads,self.heads_dim)
        keys=self.keys(keys)
        values=self.values(values)
        query=self.query(query)
        #==============================================================
        energy=torch.einsum("nqhd,nkhd->nhqk",[queries,keys])# dot product
        #queries.shape=(N,query_len,heads,embedding_dim)
        #keys.shape= (N,keys_len,heads,embedding_dim)
        #energy.shape=(N,heads,qery_len,keys_len)
        #==============================================================
        if mask is not None:
            energy=energy.masked_fill(mask==0,float("-1e20"))# fill the mask with zeros and negative inf
        attention=torch.softmax(energy/torch.sqrt(self.embedding_size),dim=3)
        #create a probability of the input
        out=torch.einsum("nhql,nlhd->nqhd",[attention,values]).reshape(
        N,query_len,self,embedding_size)
        # attention shape=(N,heads,query,k_len)
        # values shape=(N,value_len,heads,heads_dim)
        #shape after einsum (N,query_len,heads,head_dim)
        #flatten the last two dims
        #=============================================================
        out=self.fc_out(out)# maps dim size to dim size
        return out
        

In [9]:
class Transformer_block(nn.Module):
    def __init__(self,embed_size,heads,dropout,forward_expansion):
        super(Transformer_block,self).__init__()
        self.attention=Self_attention(embed_size,heads)# get the attention layer
        self.norm1=nn.LayerNorm(embed_size)#layer normalization
        self.norm2=nn.LayerNorm(embed_size)#layer normalization
        #========================================================
        self.feed_forward=nn.Sequential(
        nn.Linear(embed_size,forward_expansion*embed_size),
        nn.ReLU(),
        nn.Linear(forward_expansion*embed_size,embed_size))
        self.dropout=nn.Dropout(dropout)
        
    def forward(self,value,key,query,mask):
        attention=self.attention(value,key,query,mask)
        x=self.dropout(self.norm1(attention+query)) # skip connections
        forward=self.feed_forward(x)# forward pass
        out=self.dropout(self.norm2(forward+x))# skip connection
        return out
            

In [11]:
class Encoder(nn.Module):
    def __init__(self,src_vocab_size,
                embedding_size,n_layers,
                 heads,device,forward_expansion,
                 dropout,max_length):
        super(Encoder,self).__init__()
        self.embed_size=embed_size
        self.device=device
        self.word_embedding=nn.Embedding(src_vocab_size,embed_size)
        self.postion_embedding=nn.Embedding(max_length,embed_size)
        self.layers=nn.ModuleList(
        [Transformer_block(embed_size,heads,dropout,forward_expansion)
        for _ in range(num_layers)])
        self.dropout=nn.Dropout(dropout)
    def forward(self,x,mask):
        N,seq_length=x.shape
        positions=torch.arange(0,seq_length).expand(N,seq_length).to(self.device)
        out=self.dropout(self.word_embedding(x)+self.word_embedding(positions))
        
        for layer in self.layers:
            out=layer(out,out,out,mask)# the query, keys,values are the same
        return out


In [12]:
class Decoder_block(nn.Module):
    def __init__(self,embed_size,heads,forward_expansion,dropout,device):
        super(Decoder_block,self).__init__()
        self.attention=Self_attention(embed_size,heads)
        self.norm1=nn.LayerNorm(embed_size)
        self.transformer_block=Transformer_block(
        embed_size,heads,dropout,forwards_expansion)
        self.dropout=nn.Dropout(dropout)
    #===============================================================
    def forward(self,x,value,key,src_mask,target_mask):
        attention=self.attention(x,x,x,target_mask)
        query=self.dropout(self.norm1(attention+x))
        out=self.transformer_block(value,key,query,src_mask)
        return out

In [13]:
class Decoder(nn.Module):
    def __init__(self,target_vocab_size,embed_size,num_layers,
                heads, forward_expansion, dropout,device,max_length):
        super (Decoder,self).__init__()
        self.device=device
        self.word_embedding=nn.Embedding(target_vocab_size, embed_size)
        self.postion_embedding=nn.Embedding(max_length,embed_size)
        self.layers=nn.ModuleList(
        [
            Decoder_block(embed_size,heads,forward_expansion,dropout,device)
        for _ in range(num_layers)])
        self.fc_out=nn.Linear(embed_size,target_vocab_size)
        self.dropout=nn.Dropout(dropout)
        
    def forward(self,x,enc_out,src_mask,target_mask):
        N,seq_length=x.shape
        positions=torch.arange(0,seq_length).expand(N,seq_length).to(self.device)
        x=self.dropout(self.word_embedding(x)+self.word_embedding(positions))
        
        for layer in self.layers:
            x=layer(x,enc_out,enc_out,src_mask,target_mask)# the query, keys,values are the same
        out=self.fc_out(x)
        return out

In [16]:
class Transformer(nn.Module):
    def __init__(self,
                 src_vocab_size,
                 target_vocab_size,
                 src_pad_idx,
                 target_pad_idx,
                 embed_size=256,
                 num_layers=6,
                 forward_expansion=4,
                 heads=8,
                 dropout=0,
                 device='cuda',
                 max_length=100):
        super(Transformer,self).__init__()
        self.encoder=Encoder(src_vocab_size,
                            embed_size,
                            num_layers,
                            heads,
                            device,forward_expansion,
                            dropout,
                            max_length)
        self.decoder=Decoder(target_vocab_size,embed_size,num_layers,
    heads,
    forward_expansion,
    dropout,
    device,
    max_length,)
        self.src_pad_idx=src_pad_idx
        self.target_pad_idx=target_pad_idx
        self.device=device
    def make_src_mask(self,src):
        src_msk=(src!=self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        return src_msk.to(self.device)
    
    def make_target_mask(self,target):
        N,target_len=target.shape
        target_mask=torch.tril(torch.ones((target_len,target_len))).expand(N,1,target_len,target_len)
        return target_mask.to(self.device)
    
    def forward(self,src,target):
        src_mask=make_src_mask(src)
        target_mask=make_target_mask(target)
        enc_src=self.encoder(src,src_mask)
        out=self.decoder(target,enc_src,src_mask,target_mask)
        return out